In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Model

In [2]:
train = pd.read_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/train.csv')
test = pd.read_csv('C:/Users/daniel.brooks/Documents/GitHub/Kaggle/June Playground/test.csv')

target = pd.get_dummies(train['target'])
y = train['target']

X = train.iloc[:,1:-1]
test = test.iloc[:,1:]

train.shape, test.shape

((200000, 77), (100000, 75))

In [5]:
def LSTM():
    inputs = layers.Input(shape=(75,1), name='Input')
    x = layers.LSTM(100, activation=tf.keras.activations.relu)(inputs)
    out = layers.Dense(9, activation='softmax', name='out')(x)   

    model = Model(inputs, out)
    return model

es = callbacks.EarlyStopping(
    monitor = 'val_categorical_crossentropy', 
    min_delta = 0.0000001, 
    patience = 2,
    mode = 'min',
    baseline = None, 
    restore_best_weights = True,
    verbose = 1)


plateau  = callbacks.ReduceLROnPlateau(
                monitor = 'val_categorical_crossentropy',
                factor = 0.5, 
                patience = 2, 
                mode = 'min', 
                min_delt = 0.0000001,
                cooldown = 0, 
                min_lr = 1e-7,
                verbose = 1) 

metrics = [tf.keras.metrics.CategoricalCrossentropy()]
loss = tf.keras.losses.CategoricalCrossentropy(
                from_logits=False,
                label_smoothing=0,
                reduction="auto",
                name="categorical_crossentropy")

In [6]:
N_FOLDS = 10
SEED = 2021
oof = np.zeros((X.shape[0],9))
pred = np.zeros((test.shape[0],9))


skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for fold, (tr_idx, ts_idx) in enumerate(skf.split(X, y)):
    print(f"===== FOLD {fold} =====")
       
    x_tr = X.iloc[tr_idx] 
    y_tr = target.iloc[tr_idx] 
    
    x_ts = X.iloc[ts_idx] 
    y_ts = target.iloc[ts_idx] 

    model_nn= LSTM()

    model_nn.compile(optimizer='adam',
            loss = loss ,
            metrics = metrics)
    
    model_nn.fit(x_tr,
                y_tr,
                validation_data = (x_ts, y_ts),
                batch_size = 256,
                epochs = 50,
                verbose = 1,
                callbacks = [es,plateau])
    
    
    oof[ts_idx] = model_nn.predict(x_ts)

    score = log_loss(y_ts, oof[ts_idx])
    print(f"FOLD {fold} Score {score}\n")
    
    pred += model_nn.predict(test)/ N_FOLDS

score = log_loss(target, oof)
print(f"Score total {score}\n")  

===== FOLD 0 =====
Epoch 1/50
